# Ad break 감지 및 맥락적 광고 타겟팅

맥락적 광고는 광고가 사용자가 소비하는 웹페이지나 미디어의 맥락과 매칭되는 타겟 광고의 한 형태입니다. 이 프로세스에는 세 가지 주요 참여자가 있습니다: 퍼블리셔(웹사이트 또는 콘텐츠 소유자), 광고주, 그리고 소비자입니다. 퍼블리셔는 플랫폼과 콘텐츠를 제공하고, 광고주는 맥락에 맞는 광고를 만듭니다. 소비자는 콘텐츠와 상호작용하고, 맥락을 기반으로 관련 광고가 표시되어 더 개인화되고 관련성 있는 광고 경험을 만듭니다.

맥락적 광고의 어려운 영역은 video on demand(VOD) 플랫폼에서 스트리밍하기 위한 미디어 콘텐츠에 광고를 삽입하는 것입니다. 이 프로세스는 전통적으로 인간 전문가가 콘텐츠를 분석하고, 내러티브의 breaks를 식별하고 관련 키워드나 카테고리를 할당하는 수동 태깅에 의존했습니다. 하지만 이 접근 방식은 시간이 많이 걸리고, 주관적이며, 콘텐츠의 전체 맥락이나 뉘앙스를 포착하지 못할 수 있습니다. 전통적인 AI/ML 솔루션은 이 프로세스를 자동화할 수 있지만, 종종 광범위한 훈련 데이터가 필요하고 비용이 많이 들며 기능이 제한적일 수 있습니다.

![Ad decisions](./static/images/02-ad-breaks.jpg)

대규모 언어 모델이 지원하는 Generative AI는 이 과제에 대한 유망한 솔루션을 제공합니다. 이러한 모델의 방대한 지식과 맥락적 이해를 활용함으로써, 퍼블리셔는 자동으로 미디어 자산에 대한 맥락적 인사이트와 분류법을 생성할 수 있습니다. 이 접근 방식은 프로세스를 간소화하고 정확하고 포괄적인 맥락적 이해를 제공하여 효과적인 광고 타겟팅과 미디어 아카이브의 수익화를 가능하게 합니다.

이 워크숍의 이 부분을 마치면 비디오에 대한 다음과 같은 메타데이터를 생성하게 됩니다:
* 비디오에서 사용 가능한 고품질 광고 배치 기회 또는 _breaks_ 목록
* Ad Decision Servers를 사용한 자동 배치를 위해 광고주가 콘텐츠를 분류하는 데 사용하는 IAB Content Taxonomy를 사용한 분류를 포함하여 각 break 전후의 비디오에 대한 맥락적 정보

# Prerequisites

이 노트북을 실행하려면 노트북 환경을 설정하고 오디오, 시각적, 의미론적 정보를 사용하여 비디오를 세그먼트화한 이전의 모든 기초 노트북을 실행했어야 합니다:
1. [00-prerequisites.ipynb](00-prerequisites.ipyn)
2. [01A-visual-segments-frames-shots-scenes.ipynb](01A-visual-segments-frames-shots-scenes.ipynb) 
3. [01B-audio-segments.ipynb](01B-audio-segments.ipynb) nts-frames-shots-scenes.ipynb) 
3. [01B-audio-segments.ipynb](01B-audio-segments.ipynb) 

### Import python packages

In [ ]:
from pathlib import Path
import os
import json
import json
import boto3
from botocore.exceptions import ClientError
import json_repair
import copy
from termcolor import colored
from IPython.display import JSON
from IPython.display import Video
from IPython.display import Pretty
from IPython.display import Image as DisplayImage
from lib.frames import VideoFrames
from lib.shots import Shots
from lib.scenes import Scenes
from lib.transcript import Transcript
#from lib.chapters import Chapters
from lib import frame_utils
from lib import util
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO
import copy
import time

### 이전 노트북에서 저장된 값 검색하기
이 노트북을 실행하려면 패키지 종속성을 설치하고 SageMaker 환경에서 일부 정보를 수집한 이전 노트북 00_prerequisites.ipynb를 실행했어야 합니다.




In [ ]:
store -r

# 아키텍처

이 실습 워크플로우는 SageMaker의 AWS 서비스를 사용합니다. scenes, conversation topics 및 광고 콘텐츠 분류법을 입력으로 받아 맥락적 광고 breaks와 chapter 세그먼트를 출력으로 생성합니다.

![Contextual Ads workflow with outputs](./static/images/02-contextual-ads-workflow-w-outputs-drawio.png)

# 내러티브에서 chapters를 식별하기 위해 scenes와 topics를 정렬하여 광고 배치 기회 찾기

[Video segmentation notebook](video-understanding-with-generative-ai-on-aws-main/01-video-time-segmentation.ipynb)에서, 우리는 비디오의 시각적 및 오디오 단서를 별도로 처리했습니다. 이제 이들을 함께 가져와서 transcription topics가 scenes와 정렬되도록 하는 한 단계를 더 수행할 것입니다. 진행 중인 대화나 scene 중에 광고를 삽입하는 것은 원하지 않는 마지막 일입니다. 정렬을 생성하기 위해, 시작 및 종료 타임스탬프와 주제를 요약하는 텍스트 설명으로 표현되는 각 대화 주제를 반복할 것입니다. 각 주제에 대해, 코드는 주제의 타임스탬프 범위와 겹치거나 그 안에 속하는 관련 비디오 scenes를 식별합니다. 이 프로세스의 출력은 chapters 목록이며, 각 chapter는 해당 오디오 대화와 정렬되는 비디오 scenes를 나타내는 scene ID 목록을 포함합니다. 정렬 프로세스 후에, 우리는 시각적 및 오디오 단서를 최종 chapters로 결합했습니다. chapters 사이의 breaks는 비디오 콘텐츠의 맥락적 변화 사이에서 발생하기 때문에 광고 삽입에 이상적인 위치입니다.

실제 응용에서는 이러한 breaks를 운영자에게 제안으로 제시하고 최종 광고 배치를 확인하기 위한 human-in-the-loop 단계를 갖는 것을 권장합니다.




In [ ]:
class Chapters:
    def __init__(self, topics, scenes, frames):
        self.video_asset_dir = frames.video_asset_dir()
        self.chapters = self.align_scenes_in_chapters(topics, scenes, frames)
        
    def align_scenes_in_chapters(self, topics, scenes, frames):
        """
        Aligns video scenes with conversation topics to create chronological chapters.
    
        Args:
            topics: List of conversation topics with start_ms, end_ms, and reason
            scenes: List of scene metadata with start_ms and end_ms
            frames: List of video frame metadata
    
        Returns:
            List of chapters, each containing aligned scenes and associated text
    
        Note:
            - Handles scenes without conversations
            - Merges overlapping topics
            - Preserves chronological order
            - Creates Chapter objects for each segment
    """
        scenes = copy.deepcopy(scenes)
    
        chapters = []
        for topic in topics:
            
            topic_start_ms = topic['start_ms']
            topic_end_ms = topic['end_ms']
            text = topic['reason']

            # find all the frames that align with the conversation topic
            stack = []
            while len(scenes) > 0:
                scene = scenes[0]
                frame_start = scene['start_ms']
                frame_end = scene['end_ms']

                
                if frame_start > topic_end_ms:
                    # topic overlaps scenes that belong to previous topic - merge the text
                    if not stack:
                        num_chapters = len(chapters)
                        if num_chapters > 0:
                            chapters[num_chapters-1]['text'] = chapters[num_chapters-1]['text'] + ' ' + text
                        
                    break
    
                # scenes before any conversation starts
                if frame_end < topic_start_ms:
                    chapter = Chapter(len(chapters), [scene], frames).__dict__
                    chapters.append(chapter)
                    scenes.pop(0)
                    continue
    
                stack.append(scene)
                scenes.pop(0)
    
            if stack:
                chapter = Chapter(len(chapters), stack, frames, text).__dict__
                chapters.append(chapter)
    
        ## There could be more scenes without converations, append them
        for scene in scenes:
            chapter = Chapter(len(chapters), [scene], frames).__dict__
            chapters.append(chapter)
    
        return chapters

class Chapter:
    def __init__(self, chapter_id, scenes, frames, text = ''):
        self.scene_ids = [scene['id'] for scene in scenes]
        self.start_frame_id = scenes[0]['start_frame_id']
        self.end_frame_id = scenes[-1]['end_frame_id']
        self.start_ms = scenes[0]['start_ms']
        self.end_ms = scenes[-1]['end_ms']
        self.id = chapter_id
        self.text = text
        #folder = os.path.join(frames.video_asset_dir(), 'chapters')
        #os.makedirs(folder, exist_ok=True) 
        self.composite_images = frames.create_composite_images(frames.frames[self.start_frame_id:self.end_frame_id+1], 'chapters', prefix="chapter_")
        
        return 


In [ ]:
video['chapters'] = Chapters(video['topics'], video['scenes'].scenes, video['frames'])

결과 검토합니다.

In [ ]:
display(JSON(video['chapters'].chapters))

#### chapters 시각화

이제 각 chapter의 프레임과 텍스트를 시각화해 보겠습니다. 이것들은 광고 breaks에 대한 맥락적 정보를 생성하기 위한 프롬프트의 입력이 될 것입니다. 일부 chapters에는 관련된 텍스트가 없을 수 있습니다.

<div class="alert alert-block alert-info">
💡 chapters를 보려면 출력 상자의 스크롤 바를 사용하세요. 일부 chapters는 단일 composite image에 맞출 수 있는 것보다 더 많은 프레임을 포함하므로, 각 chapter에 대해 여러 composite images가 표시될 수 있습니다.
</div>

In [ ]:
# visualize the chapters

STOP=10
for counter, b in enumerate(video["chapters"].chapters):
    print(f'\nChapter {counter}: frames {b["start_frame_id"] } to {b["end_frame_id"] }, scenes { b["scene_ids"][0] } to { b["scene_ids"][-1] }, time { b["start_ms"]} to { b["end_ms"] } =======\n')
    if len(b["text"]) > 0: 
        print(f'\nChapter Text: { b["text"] }')
    else:
        print(f'\nChapter Text (conversation topic): None')

    video['frames'].display_frames(start=b['start_frame_id'], end=b['end_frame_id']+1)

    # ALTERNATIVE: view the composite images that will be used in prompts
    #for image_file in b['composite_images']:
    #    display(DisplayImage(filename=image_file['file'], height=100))
    #if counter == STOP:
    #    break

# chapter 수준의 맥락적 정보 생성

마지막 단계는 시각적으로 그리고 오디오로 정렬된 데이터를 Claude 3 Sonnet에 보내 각 주제에 대한 맥락적 정보를 생성하는 것입니다. 이는 Claude 3.5 제품군 모델의 멀티모달 기능을 활용하는 접근 방식입니다. 우리의 테스트에서, 이러한 모델들은 적절한 지침이 제공될 때 큰 이미지에서 세부 사항을 포착하고 이미지 시퀀스를 따를 수 있는 능력을 보여주었습니다.

Claude3.5 Sonnet의 입력을 준비하기 위해, 먼저 각 주제와 관련된 비디오 프레임을 조합하고 composite image 그리드를 만듭니다. 실험을 통해, 우리는 7행 4열의 이미지 그리드 비율이 최적임을 발견했습니다. 이는 각 개별 프레임 타일에서 충분한 세부 사항을 유지하면서도 Claude의 5MB 이미지 파일 크기 제한 아래에 맞는 1568 x 1540 픽셀 이미지를 구성할 것입니다. 또한 필요한 경우 여러 이미지를 조합할 수도 있습니다.

이후, composite images, transcription, IAB Content taxonomy 정의, GARM taxonomy 정의가 Claude3 Haiku 모델에 대한 단일 쿼리로 설명, 감정, IAB taxonomy, GARM taxonomy 및 기타 관련 정보를 생성하기 위한 프롬프트에 입력됩니다. 뿐만 아니라, 이 접근 방식을 매번 모델을 훈련할 필요 없이 모든 분류법이나 사용자 지정 레이블링 사용 사례에 적용할 수 있습니다. 이것이 이 접근 방식의 진정한 힘이 있는 곳입니다. 필요한 경우 최종 출력을 인간 검토자에게 제시하여 최종 확인을 받을 수 있습니다. 다음은 특정 주제에 대한 composite image 그리드와 해당 맥락적 출력의 예시입니다.

![Contextualized chapters](./static/images/02-chapter-contextualization.png)

## IAB Content Taxonomy 정의 다운로드

IAB(Interactive Advertising Bureau) Taxonomy는 디지털 광고 콘텐츠와 대상을 위한 표준화된 분류 시스템입니다. 디지털 콘텐츠를 분류하기 위한 계층적 구조를 제공하여 광고주와 퍼블리셔가 디지털 광고를 구성, 타겟팅 및 측정하기 쉽게 만듭니다.

Anthropic Claude에게 이 분류법을 사용하여 chapters를 분류하도록 지시하여 서로 다른 chapters 사이에 맞을 수 있는 광고 종류를 식별하는 데 도움을 줄 것입니다.


In [ ]:
iab_file = 'iab_content_taxonomy_v3.json'
url = f"https://dx2y1cac29mt3.cloudfront.net/iab/{iab_file}"

!curl {url} -o {iab_file}

In [ ]:
def load_iab_taxonomy(file):
    """
    Loads IAB taxonomy definitions from a JSON file.
    Args:
        file: Path to the IAB taxonomy JSON file
    Returns:
        Dictionary containing IAB taxonomy definitions
    """
    with open(file) as f:
        iab_taxonomies = json.load(f)
    return iab_taxonomies

In [ ]:
iab_taxonomy = load_iab_taxonomy(iab_file)
display(JSON(iab_taxonomy))

## 각 chapter 세그먼트에 대한 맥락적 메타데이터를 생성하기 위한 프롬프트 구성

이 예시는 Foundation 모델과의 다중 턴 대화를 시뮬레이션하는 이 프롬프트를 위해 Amazon Bedrock과 함께 [Anthropic Claude Messages API (aka Conversations API)](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html)를 사용합니다.

먼저, 프롬프트의 부분들에 대한 텍스트를 생성하기 위한 헬퍼 함수를 만들어 보겠습니다.

In [ ]:
# Constructors for parts of prompt messages

def make_iab_taxonomoies(iab_list):
    iab = [item['name'] for item in iab_list]
    iab.append('None')

    return iab

def make_image_message(composite_images):
    """
    Converts a list of image files into a formatted message with base64-encoded images.
    Args:
        composite_images: List of dicts containing image file paths
    Returns:
        Dict with 'role' and 'content' containing:
        - Text description of number of images
        - List of base64-encoded images with metadata
    """
    # adding the composite image sequences
    image_contents = [{
        'type': 'text',
        'text': 'Here are {0} images containing frame sequence that describes a scene.'.format(len(composite_images))
    }]

    open_images = []
    for image in composite_images:
        with open(image['file'], "rb") as image_file:
            image_data = image_file.read()
            open_images.append(image_file)
        image_pil = Image.open(BytesIO(image_data))
        bas64_image = frame_utils.image_to_base64(image_pil)
        image_contents.append({
            'type': 'image',
            'source': {
                'type': 'base64',
                'media_type': 'image/jpeg',
                'data': bas64_image
            }
        })

    # close the images
    for image in open_images:
        image.close()

    return {
        'role': 'user',
        'content': image_contents
    }

def make_output_example():
    """
    Creates a template message for AI model output formatting.
    Returns:
        Dict with 'role' and 'content' keys containing example JSON structure for:
        - Scene description
        - Sentiment analysis
        - IAB and GARM taxonomies
        - Brand/logo detection
        - Relevant tags
    Note:
        Used as part of the prompt to ensure consistent response formatting
    """
    example = {
        'description': {
            'text': 'The scene describes...',
            'score': 98
        },
        'sentiment': {
            'text': 'Positive',
            'score': 90
        },
        'iab_taxonomy': {
            'text': 'Station Wagon',
            'score': 80
        },
        'garm_taxonomy': {
            'text': 'Online piracy',
            'score': 90
        },
        'brands_and_logos': [
            {
                'text': 'Amazon',
                'score': 95
            },
            {
                'text': 'Nike',
                'score': 85
            }
        ],
        'relevant_tags': [
            {
                'text': 'auto racing',
                'score': 95
            }
        ]            
    }
    
    return {
        'role': 'user',
        'content': 'Return JSON format. An example of the output:\n{0}\n'.format(json.dumps(example))
    }

다음 코드 블록은 헬퍼 함수를 사용하여 프롬프트를 구성하고 추론을 수행하기 위해 Amazon Bedrock을 호출합니다.

In [ ]:
# MODEL_ID = 'anthropic.claude-3-5-sonnet-20240620-v1:0'
# MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"
MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"

MODEL_PRICING = (0.003, 0.015)
MODEL_VER = 'bedrock-2023-05-31'

def get_chapter_description(images, text, iab_definitions):
    """
    Generates chapter descriptions using image analysis, text, and IAB classifications.
    Args:
        images: List of image analysis results (max 19 images)
        text: Transcribed conversation/text (optional)
        iab_definitions: IAB taxonomy definitions with tier1 classifications
    Returns:
        Dict containing chapter description, IAB classifications, and sentiment analysis
    Note:
        - Uses Claude model for analysis
        - Requires make_iab_taxonomies(), make_output_example(), make_image_message()
        - Implements retry logic for failed inference calls
    """

      
    system = '''You are a media operation engineer. Your job is to review a clip from a video 
    content presented in a sequence of consecutive images. Each image
    contains a sequence of frames presented in a 4x7 grid reading from left to
    right and then from top to bottom. Interpret the frames as the time 
    progression of a video clip.  Don't refer to specific frames, instead, think
    about what is happening over time in the scene.  You may also optionally be given the
    conversation of the scene you can use to understand the context of
    the scene. 

    You are asked to provide the following information: a detailed 
    description to describe the scene using the visual and audio, identify the most relevant IAB taxonomy, 
    GARM, sentiment, and brands and logos that 
    may appear in the scene, and five most relevant tags from the scene.
    
    It is important to return the results in JSON format and also includes a
    confidence score from 0 to 100. Skip any explanation. Answer in detail in Korean.
    '''

    other_information = []
    other_information.append(
        {
            "type": "text",
            "text": f'''
                    Here is a list of IAB Taxonomies in <iab> tag. Only answer 
                    the IAB taxonomy from this list:
                    <iab>
                    { json.dumps(make_iab_taxonomoies(iab_definitions['tier1'])) }
                    </iab>
                    '''
        })
    
    other_information.append(
        {
            "type": "text",
            "text": f'''
                    Here is a list of GARM Taxonomies in <garm> tag. Only answer
                    the GARM taxonomy from this list:
                    <garm>
                    [
                        'Adult & Explicit Sexual Content',
                        'Arms & Ammunition',
                        'Crime & Harmful acts to individuals and Society, Human Right Violations',
                        'Death, Injury or Military Conflict',
                        'Online piracy',
                        'Hate speech & acts of aggression',
                        'Obscenity and Profanity, including language, gestures, and explicitly gory, graphic or repulsive content intended to shock and disgust',
                        'Illegal Drugs, Tobacco, ecigarettes, Vaping, or Alcohol',
                        'Spam or Harmful Content',
                        'Terrorism',
                        'Debated Sensitive Social Issue',
                        'None',
                    ]
                    </garm>
                    '''
        })

    other_information.append(
        {
            "type": "text",
            "text": f'''
                Here is a list of Sentiments in <sentiment> tag. Only answer the
                sentiment from this list:

                <sentiment>
                ['Positive', 'Neutral', 'Negative', 'None']
                </sentiment>
                '''
        })

    output_format_message = make_output_example()

    messages = []
 
    # adding sequences of composite images to the prompt.  Limit is 20.
    message_images = make_image_message(images[:19])
    messages.append(message_images)

    # adding the conversation to the prompt
    messages.append({
        'role': 'assistant',
        'content': 'Got the images. Do you have the conversation of the scene?'
    })

    message_conversation = {
        'role': 'user',
        'content': 'No conversation.'
    }
    if text:
        message_conversation['content'] = f'''
            Here is the conversation of the scene in <conversation> tag.
            <conversation>
            { text }
            </conversation>
            '''

    messages.append(message_conversation)

    # other information
    messages.append({
        'role': 'assistant',
        'content': 'OK. Do you have other information to provdie?'
    })

    messages.append({
        'role': 'user',
        'content': other_information
    })

    # output format
    messages.append({
        'role': 'assistant',
        'content': 'OK. What output format?'
    })
    messages.append(output_format_message)

    # prefill '{'
    messages.append({
        'role': 'assistant',
        'content': '{'
    })
    
    model_params = {
        'anthropic_version': MODEL_VER,
        'max_tokens': 4096,
        'temperature': 0.1,
        'top_p': 0.7,
        'top_k': 20,
        'stop_sequences': ['\n\nHuman:'],
        'system': system,
        'messages': messages
    }
    
    try:
        response = inference(model_params)
    
    except Exception as e:
        print(colored(f"ERR: inference: {str(e)}\n RETRY...", 'red'))
        response = inference(model_params)

    return response

def display_prompt(model_params):
    """
    Displays the model parameters including system prompt and messages for debugging or logging purposes.
    Args:
        model_params (dict): Dictionary containing model parameters with:
            - system (str): The system prompt text
            - messages (list): List of message objects to be displayed       
    Returns:
        None: This function prints to console and doesn't return any value
    """
    print (f'MODEL_ID: {MODEL_ID}\n')
    print (f'System Prompt:\n\n{model_params["system"]}')
    print (f'Messages:\n\n')
    for message in model_params['messages']:
        print (json.dumps(message))

    print('\n')

    return

def inference(model_params):
    """
    Invokes an Amazon Bedrock model for inference using the specified parameters.

    Args:
        model_params (dict): Parameters for the model inference, including:
            - Any model-specific parameters required for the inference call
            - Must be JSON-serializable

    Returns:
        dict: The processed response containing:
            - content (list): List of response contents where each item contains:
                - text (str): Raw text response from the model
                - json (dict): Parsed JSON response (if successful)
            - model_params (dict): Original input parameters
            - Additional response metadata from the model
    """
    model_id = MODEL_ID
    accept = 'application/json'
    content_type = 'application/json'

    bedrock_runtime_client = boto3.client(service_name='bedrock-runtime')

    response = bedrock_runtime_client.invoke_model(
        body=json.dumps(model_params),
        modelId=model_id,
        accept=accept,
        contentType=content_type
    )

    response_body = json.loads(response.get('body').read())

    # patch the json string output with '{' and parse it
    response_content = response_body['content'][0]['text']
    if response_content[0] != '{':
        response_content = '{' + response_content

    try:
        response_content = json.loads(response_content)
    except Exception as e:
        print(colored("Malformed JSON response. Try to repair it...", 'red'))
        try:
            response_content = json_repair.loads(response_content, strict=False)
        except Exception as e:
            print(colored("Failed to repair the JSON response...", 'red'))
            print(colored(response_content, 'red'))
            raise e

    response_body['content'][0]['json'] = response_content
    response_body['model_params'] = model_params

    return response_body



def display_contextual_cost(usage):
    """
    Calculate and display the estimated cost of using the model based on input and output tokens.
    Args:
        usage (dict): A dictionary containing token usage information with keys:
            - input_tokens (int): Number of input tokens used
            - output_tokens (int): Number of output tokens generated
    Returns:
        dict: A dictionary containing cost calculation details:
            - input_per_1k (float): Cost per 1000 input tokens
            - output_per_1k (float): Cost per 1000 output tokens
            - input_tokens (int): Number of input tokens used
            - output_tokens (int): Number of output tokens generated
            - estimated_cost (float): Total estimated cost in USD
    """
    # us-east-1 pricing
    input_per_1k, output_per_1k = MODEL_PRICING

    input_tokens = usage['input_tokens']
    output_tokens = usage['output_tokens']

    contextual_cost = (
        input_per_1k * input_tokens +
        output_per_1k * output_tokens
    ) / 1000

    print('\n')
    print('========================================================================')
    print('Estimated cost:', colored(f"${round(contextual_cost, 4)}", 'green'), f"in us-east-1 region with {colored(input_tokens, 'green')} input tokens and {colored(output_tokens, 'green')} output tokens.")
    print('========================================================================')

    return {
        'input_per_1k': input_per_1k,
        'output_per_1k': output_per_1k,
        'input_tokens': input_tokens,
        'output_tokens': output_tokens,
        'estimated_cost': contextual_cost,
    }


## 모든 chapter 세그먼트에 대해 프롬프트 실행

In [ ]:
total_usage = {
    'input_tokens': 0,
    'output_tokens': 0,
}

iab_definitions = load_iab_taxonomy(iab_file)

for chapter in video['chapters'].chapters:

    composite_images = chapter['composite_images']
    num_images = len(composite_images)

    chapter_id = chapter['id']
    text = chapter['text'] 

    contextual_response = get_chapter_description(composite_images, chapter['text'], iab_definitions)
    time.sleep(5)
    usage = contextual_response['usage']
    contextual = contextual_response['content'][0]['json']

    # save the contextual to the chapter
    chapter['contextual'] = {
        'usage': usage,
        **contextual
    }

    total_usage['input_tokens'] += usage['input_tokens']
    total_usage['output_tokens'] += usage['output_tokens']

    print(f"==== Chapter #{chapter['id']:02d}: Contextual information ======")
    video['frames'].display_frames(start=chapter['start_frame_id'], end=chapter['end_frame_id']+1)
    for key in ['description', 'sentiment', 'iab_taxonomy', 'garm_taxonomy']:
        print(f"{key.capitalize()}: {colored(contextual[key]['text'], 'green')} ({contextual[key]['score']}%)")

    for key in ['brands_and_logos', 'relevant_tags']:
        items = ', '.join([item['text'] for item in contextual[key]])
        if len(items) == 0:
            items = 'None'
        print(f"{key.capitalize()}: {colored(items, 'green')}")
    print(f"================================================\n\n")

output_file = os.path.join(video["output_dir"], 'scenes_in_chapters.json')
util.save_to_file(output_file, video['chapters'].chapters)

contextual_cost = display_contextual_cost(total_usage)

## Ad breaks

이 시점에서, 우리는 scenes 사이에 명확한 시각적 breaks가 있는 비디오 세그먼트를 생성했고, scenes를 오디오의 speech에서 주제 사이에 명확한 breaks가 있는 chapters로 그룹화했습니다. chapters 사이의 breaks는 모두 광고 배치 기회 후보입니다. 우리는 breaks에 어떤 광고를 배치할지 더 나은 결정을 내리기 위해 breaks에 인접한 chapter 세그먼트의 IAB taxonomy를 사용할 수 있습니다.

🤔 chapter 세그먼트를 보면서, 자신을 브랜드를 광고하고 싶은 회사라고 상상해보세요. 브랜드 안전성 측면에서 다른 breaks보다 선호하는 breaks가 있나요?

🤔 이제 자신을 시청자라고 상상해보세요. 이 제목을 선택했다면 어떤 제품이 흥미로울까요?

실제로는 광고 breaks는 소비자, 퍼블리셔, 광고주의 요구 사항을 고려하는 가치 함수에 의해 순위가 매겨질 것입니다.

## 광고 breaks 시각화

이 섹션에서는 광고 경험을 시각화하기 위해 breaks 중 하나에 테스트 광고를 삽입할 것입니다. BREAK_CHAPTER_ID 값을 변경하여 다른 chapter breaks를 시도해볼 수 있습니다.

In [ ]:
import moviepy
from moviepy.editor import VideoFileClip, concatenate_videoclips

BREAK_CHAPTER_ID = 8

ad_demo_file= f"ad_break_{ BREAK_CHAPTER_ID }_demo.mp4"
adbreak_start = video['chapters'].chapters[BREAK_CHAPTER_ID]['start_ms']/1000

clip1 = VideoFileClip(video["path"], target_resolution=(360, 640)).subclip(adbreak_start-10, adbreak_start)
clip2 = VideoFileClip("static/images/CountdownClock_0.mp4", target_resolution=(360, 640))
clip3 = VideoFileClip(video["path"], target_resolution=(360, 640)).subclip(adbreak_start, adbreak_start+10)
final_clip = concatenate_videoclips([clip1,clip2,clip3], method="compose")
final_clip.write_videofile(ad_demo_file)

In [ ]:
Video(url=ad_demo_file, width=640, height=360)



# 다음은 무엇인가요?

다른 사용 사례를 시도해보거나, 완료했다면 [Additional Resources](09-resources.ipynb) 실습으로 계속 진행할 수 있습니다.